<a href="https://colab.research.google.com/github/prathamparakh/Zephyr-Stock-Exchange-Game/blob/main/Zephyr_Stock_Exchane_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: Create a complete Python implementation of the "Zephyr Stock Exchange" strategic stock market simulation game with the following specifications:
# ## Game Overview
# - Strategic stock market simulation for 2-6 players
# - Players maximize net worth through buying/selling stocks and using event cards
# - Turn-based gameplay with bargain sessions containing 3 trading rounds each
# ## Core Game Components
# ### 1. Player Setup
# - Each player starts with ₹6,00,000 cash
# - Track individual portfolios (cash + stock holdings)
# - Player turn management system
# ### 2. Stock System
# - Initialize stocks with predefined opening prices:
#   - HDFC: ₹25
#   - Infosys: ₹80
#   - TCS: ₹40
#   - Reliance: ₹75
#   - ONGC: ₹55
#   - Wockhardt: ₹20
# - Stock price updates only at end of bargain sessions
# - Handle stock insolvency scenarios
# ### 3. Event Cards System
# - Deal 6-10 event cards per player at start of each bargain session
# - Event cards affect stock prices (e.g., "+₹15 to HDFC", "-₹30 to Infosys")
# - Cards are revealed simultaneously after 3 trading rounds
# - Calculate net impact on each stock from all revealed cards
# ### 4. Special Cards (one-time use)
# - **Loan Stock Mature**: Instantly get ₹1,00,000
# - **Right Issue**: Buy 50% more shares of target stock at ₹10 each
# - **Debenture**: Sell insolvent company stock at original opening price
# - **Stock Suspended**: Freeze stock price to previous session value
# ### 5. Game Flow Management
# - Bargain sessions with 3 trading rounds each
# - Turn-based trading (buy/sell stocks, use special cards)
# - Price updates after card reveals
# - Win condition: highest net worth after preset sessions
# ## Implementation Requirements
# ### Classes Needed:
# 1. **Player** class with cash, stocks, event cards, special cards
# 2. **Stock** class with name, current price, opening price, price history
# 3. **EventCard** class with stock target and price impact
# 4. **SpecialCard** class with card type and effects
# 5. **Game** class managing

import random

class Stock:
    def __init__(self, name, opening_price):
        self.name = name
        self.opening_price = opening_price
        self.current_price = opening_price
        self.price_history = [opening_price]
        self.is_insolvent = False
        self.is_suspended = False

    def update_price(self, net_impact):
        if self.is_suspended:
            return

        new_price = self.current_price + net_impact
        if new_price <= 0:
            self.current_price = 0
            self.is_insolvent = True
            print(f"{self.name} is now insolvent!")
        else:
            self.current_price = new_price
        self.price_history.append(self.current_price)

    def suspend(self):
        self.is_suspended = True
        print(f"{self.name} stock is now suspended.")

    def unsuspend(self):
        self.is_suspended = False
        print(f"{self.name} stock is no longer suspended.")

    def __str__(self):
        return f"{self.name}: ₹{self.current_price:.2f} (Opening: ₹{self.opening_price:.2f})"


class EventCard:
    def __init__(self, stock_name, price_impact):
        self.stock_name = stock_name
        self.price_impact = price_impact

    def __str__(self):
        sign = '+' if self.price_impact >= 0 else ''
        return f"{sign}₹{self.price_impact} to {self.stock_name}"


class SpecialCard:
    def __init__(self, card_type):
        self.card_type = card_type

    def __str__(self):
        return self.card_type

class Player:
    def __init__(self, name):
        self.name = name
        self.cash = 600000
        self.portfolio = {}  # {stock_name: quantity}
        self.event_cards = []
        self.special_cards = []

    def buy_stock(self, stock, quantity):
        cost = stock.current_price * quantity
        if self.cash >= cost:
            self.cash -= cost
            self.portfolio[stock.name] = self.portfolio.get(stock.name, 0) + quantity
            print(f"{self.name} bought {quantity} shares of {stock.name} for ₹{cost:.2f}.")
            return True
        else:
            print(f"{self.name} doesn't have enough cash to buy {quantity} shares of {stock.name}.")
            return False

    def sell_stock(self, stock, quantity):
        if self.portfolio.get(stock.name, 0) >= quantity:
            value = stock.current_price * quantity
            self.cash += value
            self.portfolio[stock.name] -= quantity
            if self.portfolio[stock.name] == 0:
                del self.portfolio[stock.name]
            print(f"{self.name} sold {quantity} shares of {stock.name} for ₹{value:.2f}.")
            return True
        else:
            print(f"{self.name} doesn't own {quantity} shares of {stock.name}.")
            return False

    def use_special_card(self, card, game):
        if card not in self.special_cards:
            print(f"{self.name} doesn't have the '{card.card_type}' card.")
            return False

        if card.card_type == "Loan Stock Mature":
            self.cash += 100000
            print(f"{self.name} used 'Loan Stock Mature' and gained ₹1,00,000.")
            self.special_cards.remove(card)
            return True

        elif card.card_type == "Right Issue":
            # Need to specify target stock
            target_stock_name = input(f"{self.name}, which stock do you want to use 'Right Issue' on? ")
            target_stock = game.get_stock_by_name(target_stock_name)

            if target_stock:
                current_holding = self.portfolio.get(target_stock_name, 0)
                additional_shares = current_holding // 2 # 50% more shares
                if additional_shares > 0:
                    cost = additional_shares * 10
                    if self.cash >= cost:
                        self.cash -= cost
                        self.portfolio[target_stock_name] = self.portfolio.get(target_stock_name, 0) + additional_shares
                        print(f"{self.name} used 'Right Issue' on {target_stock_name} and bought {additional_shares} shares at ₹10 each.")
                        self.special_cards.remove(card)
                        return True
                    else:
                         print(f"{self.name} doesn't have enough cash to buy {additional_shares} shares at ₹10 each.")
                         return False
                else:
                    print(f"{self.name} doesn't own enough {target_stock_name} shares to use 'Right Issue'.")
                    return False
            else:
                print(f"Invalid stock name: {target_stock_name}.")
                return False

        elif card.card_type == "Debenture":
            # Need to specify target insolvent stock
            insolvent_stocks = [s for s in game.stocks if s.is_insolvent]
            if not insolvent_stocks:
                print("There are no insolvent stocks to use 'Debenture' on.")
                return False

            print("Insolvent stocks available:")
            for s in insolvent_stocks:
                print(f"- {s.name}")

            target_stock_name = input(f"{self.name}, which insolvent stock do you want to use 'Debenture' on? ")
            target_stock = game.get_stock_by_name(target_stock_name)

            if target_stock and target_stock.is_insolvent:
                if self.portfolio.get(target_stock_name, 0) > 0:
                    quantity_to_sell = self.portfolio.get(target_stock_name, 0)
                    value = target_stock.opening_price * quantity_to_sell
                    self.cash += value
                    del self.portfolio[target_stock_name]
                    print(f"{self.name} used 'Debenture' on {target_stock_name} and sold {quantity_to_sell} shares at opening price ₹{value:.2f}.")
                    self.special_cards.remove(card)
                    return True
                else:
                    print(f"{self.name} doesn't own any shares of the insolvent stock {target_stock_name}.")
                    return False
            elif target_stock and not target_stock.is_insolvent:
                 print(f"{target_stock_name} is not insolvent.")
                 return False
            else:
                print(f"Invalid stock name: {target_stock_name}.")
                return False


        elif card.card_type == "Stock Suspended":
             # Need to specify target stock
            target_stock_name = input(f"{self.name}, which stock do you want to suspend? ")
            target_stock = game.get_stock_by_name(target_stock_name)

            if target_stock:
                target_stock.suspend()
                self.special_cards.remove(card)
                return True
            else:
                print(f"Invalid stock name: {target_stock_name}.")
                return False


        else:
            print(f"Unknown special card type: {card.card_type}")
            return False


    def calculate_net_worth(self, stocks):
        stock_value = sum(self.portfolio.get(stock.name, 0) * stock.current_price for stock in stocks)
        return self.cash + stock_value

    def __str__(self):
        portfolio_str = ", ".join([f"{qty}x {name}" for name, qty in self.portfolio.items()]) if self.portfolio else "None"
        return f"{self.name}: Cash ₹{self.cash:.2f}, Portfolio: {portfolio_str}"


class Game:
    def __init__(self, players_names, num_sessions=5):
        if not 2 <= len(players_names) <= 6:
            raise ValueError("Number of players must be between 2 and 6.")

        self.players = [Player(name) for name in players_names]
        self.stocks = [
            Stock("HDFC", 25),
            Stock("Infosys", 80),
            Stock("TCS", 40),
            Stock("Reliance", 75),
            Stock("ONGC", 55),
            Stock("Wockhardt", 20)
        ]
        self.event_card_deck = self._create_event_card_deck()
        self.special_card_deck = self._create_special_card_deck()
        self.num_sessions = num_sessions
        self.current_session = 0
        self.current_round = 0
        self.current_player_index = 0
        self.revealed_event_cards = [] # Cards revealed in the current session

    def _create_event_card_deck(self):
        # Define a set of possible event card impacts
        impacts = [-30, -20, -15, -10, -5, 5, 10, 15, 20, 30]
        cards = []
        for stock in self.stocks:
            # Create a variety of cards for each stock
            cards.extend([EventCard(stock.name, impact) for impact in impacts])
        random.shuffle(cards)
        return cards

    def _create_special_card_deck(self):
        # Define the special cards with a certain frequency
        cards = [SpecialCard("Loan Stock Mature")] * 3 + \
                [SpecialCard("Right Issue")] * len(self.stocks) + \
                [SpecialCard("Debenture")] * 2 + \
                [SpecialCard("Stock Suspended")] * 2
        random.shuffle(cards)
        return cards

    def get_stock_by_name(self, name):
        for stock in self.stocks:
            if stock.name.lower() == name.lower():
                return stock
        return None

    def deal_cards_to_players(self):
        num_event_cards_per_player = random.randint(6, 10)
        print(f"\nDealing {num_event_cards_per_player} event cards and 1 special card to each player...")
        for player in self.players:
            player.event_cards = random.sample(self.event_card_deck, num_event_cards_per_player)
            self.event_card_deck = [card for card in self.event_card_deck if card not in player.event_cards]

            if self.special_card_deck:
                 special_card = self.special_card_deck.pop(0)
                 player.special_cards.append(special_card)
            else:
                 print(f"Not enough special cards left to deal to {player.name}.")


    def display_game_state(self):
        print("\n--- Current Game State ---")
        print(f"Session: {self.current_session}/{self.num_sessions}, Round: {self.current_round}/3")
        print("\nStock Prices:")
        for stock in self.stocks:
            print(f"- {stock}")
        print("\nPlayers:")
        for player in self.players:
            print(f"- {player}")
        print("--------------------------")

    def player_turn(self, player):
        print(f"\n--- {player.name}'s Turn (Cash: ₹{player.cash:.2f}) ---")

        while True:
            print("\nAvailable Actions:")
            print("1. Buy Stock")
            print("2. Sell Stock")
            print("3. Use Special Card")
            print("4. View Portfolio")
            print("5. View Event Cards")
            print("6. End Turn")

            choice = input("Enter your choice (1-6): ")

            if choice == '1':
                print("\nAvailable Stocks:")
                for i, stock in enumerate(self.stocks):
                    if not stock.is_insolvent and not stock.is_suspended:
                       print(f"{i+1}. {stock}")

                stock_choice = input("Enter number of stock to buy: ")
                try:
                    stock_index = int(stock_choice) - 1
                    if 0 <= stock_index < len(self.stocks):
                        chosen_stock = self.stocks[stock_index]
                        if chosen_stock.is_insolvent or chosen_stock.is_suspended:
                             print("Cannot buy this stock right now.")
                             continue
                        quantity_str = input(f"How many shares of {chosen_stock.name} do you want to buy? ")
                        try:
                            quantity = int(quantity_str)
                            if quantity > 0:
                                player.buy_stock(chosen_stock, quantity)
                            else:
                                print("Quantity must be positive.")
                        except ValueError:
                            print("Invalid quantity.")
                    else:
                        print("Invalid stock number.")
                except ValueError:
                    print("Invalid input.")

            elif choice == '2':
                if not player.portfolio:
                    print("Your portfolio is empty.")
                    continue

                print("\nYour Portfolio:")
                portfolio_items = list(player.portfolio.items())
                for i, (stock_name, quantity) in enumerate(portfolio_items):
                    print(f"{i+1}. {stock_name}: {quantity} shares")

                sell_choice = input("Enter number of stock to sell: ")
                try:
                    sell_index = int(sell_choice) - 1
                    if 0 <= sell_index < len(portfolio_items):
                        stock_name_to_sell, current_quantity = portfolio_items[sell_index]
                        stock_to_sell = self.get_stock_by_name(stock_name_to_sell)
                        if stock_to_sell.is_suspended:
                             print("Cannot sell this stock right now as it is suspended.")
                             continue
                        quantity_str = input(f"How many shares of {stock_name_to_sell} do you want to sell? (You own {current_quantity}) ")
                        try:
                            quantity = int(quantity_str)
                            if 0 < quantity <= current_quantity:
                                player.sell_stock(stock_to_sell, quantity)
                            else:
                                print("Invalid quantity.")
                        except ValueError:
                            print("Invalid quantity.")
                    else:
                        print("Invalid stock number.")
                except ValueError:
                    print("Invalid input.")

            elif choice == '3':
                if not player.special_cards:
                    print("You have no special cards.")
                    continue

                print("\nYour Special Cards:")
                for i, card in enumerate(player.special_cards):
                    print(f"{i+1}. {card}")

                card_choice = input("Enter number of card to use (or 0 to cancel): ")
                try:
                    card_index = int(card_choice) - 1
                    if card_index == -1:
                         continue
                    if 0 <= card_index < len(player.special_cards):
                        chosen_card = player.special_cards[card_index]
                        player.use_special_card(chosen_card, self) # Pass game object for card effects
                    else:
                        print("Invalid card number.")
                except ValueError:
                    print("Invalid input.")

            elif choice == '4':
                print("\nYour Portfolio:")
                if player.portfolio:
                    for stock_name, quantity in player.portfolio.items():
                         print(f"- {stock_name}: {quantity} shares")
                else:
                     print("Empty.")
                print(f"Cash: ₹{player.cash:.2f}")


            elif choice == '5':
                 print("\nYour Event Cards:")
                 if player.event_cards:
                     for i, card in enumerate(player.event_cards):
                         print(f"{i+1}. {card}")
                 else:
                     print("You have no event cards.")

            elif choice == '6':
                print(f"{player.name} ends their turn.")
                break

            else:
                print("Invalid choice. Please try again.")


    def reveal_event_cards(self):
        print("\n--- Revealing Event Cards ---")
        all_revealed_cards = []
        for player in self.players:
            # Players choose which cards to reveal (optional rule, for simplicity just reveal all dealt)
            # For now, all event cards dealt in the session are revealed
            all_revealed_cards.extend(player.event_cards)
            player.event_cards = [] # Cards are discarded after revealing

        self.revealed_event_cards = all_revealed_cards # Store for summary

        net_impact_per_stock = {stock.name: 0 for stock in self.stocks}
        for card in all_revealed_cards:
            print(f"Card revealed: {card}")
            if card.stock_name in net_impact_per_stock:
                net_impact_per_stock[card.stock_name] += card.price_impact

        print("\nNet Impact on Stocks:")
        for stock_name, impact in net_impact_per_stock.items():
            print(f"- {stock_name}: {impact}")

        print("\nUpdating Stock Prices...")
        for stock in self.stocks:
            if not stock.is_insolvent and not stock.is_suspended:
                 stock.update_price(net_impact_per_stock.get(stock.name, 0))

    def check_insolvency_and_debenture(self):
        insolvent_stocks_this_session = [stock for stock in self.stocks if stock.is_insolvent and stock.current_price == 0 and stock.price_history[-2] > 0] # Just became insolvent

        if insolvent_stocks_this_session:
             print("\n--- Insolvency Check ---")
             for stock in insolvent_stocks_this_session:
                 print(f"{stock.name} became insolvent this session.")

             for player in self.players:
                 if any(card.card_type == "Debenture" for card in player.special_cards):
                      debenture_card = next((card for card in player.special_cards if card.card_type == "Debenture"), None)
                      if debenture_card:
                          # Player has a Debenture card, offer to use it
                          while True:
                              use_debenture = input(f"{player.name}, do you want to use your 'Debenture' card on an insolvent stock? (yes/no): ").lower()
                              if use_debenture == 'yes':
                                  player.use_special_card(debenture_card, self)
                                  break # Assume they use only one per session on newly insolvent stocks
                              elif use_debenture == 'no':
                                  break
                              else:
                                  print("Invalid input. Please enter 'yes' or 'no'.")

        # Handle forced sale of insolvent stock for players without Debenture or if they choose not to use it
        for player in self.players:
            for stock_name in list(player.portfolio.keys()): # Iterate over a copy since we might modify
                 stock = self.get_stock_by_name(stock_name)
                 if stock and stock.is_insolvent and player.portfolio.get(stock_name, 0) > 0:
                     print(f"\n{player.name}, you own {player.portfolio[stock_name]} shares of insolvent stock {stock_name}.")
                     if not any(card.card_type == "Debenture" for card in player.special_cards):
                          # Player does not have a Debenture card
                         print(f"You do not have a 'Debenture' card. Your {stock_name} shares are now worthless.")
                         del player.portfolio[stock_name]
                     # If they had a debenture but didn't use it, their shares are also worthless
                     elif stock_name in [s.name for s in insolvent_stocks_this_session]: # Only force sell if it became insolvent *this* session and debenture wasn't used on it
                         print(f"Your {stock_name} shares are now worthless as you did not use your 'Debenture' card on it.")
                         del player.portfolio[stock_name]



    def check_suspended_stocks(self):
         print("\n--- Checking Suspended Stocks ---")
         for stock in self.stocks:
             if stock.is_suspended:
                 print(f"{stock.name} is still suspended. Price is frozen.")
             # Rule: Suspension lasts for 1 session. Unsuspend at the start of the *next* session.
             # This is handled implicitly by not suspending them again next session unless another card is played.
             # If we wanted 1 session suspension, we'd need a flag on the stock or track in Game.

    def play_session(self):
        self.current_session += 1
        print(f"\n======== Starting Session {self.current_session}/{self.num_sessions} ========")

        # Unsuspend stocks from previous session (if any)
        for stock in self.stocks:
             if stock.is_suspended:
                 stock.unsuspend()


        self.deal_cards_to_players()

        for self.current_round in range(1, 4): # 3 trading rounds
            print(f"\n--- Starting Trading Round {self.current_round} ---")
            for player in self.players:
                self.player_turn(player)
                self.display_game_state() # Show state after each player's turn


        self.reveal_event_cards()
        self.check_insolvency_and_debenture()


        print(f"\n======== Session {self.current_session} Ended ========")
        self.display_game_state()


    def end_game(self):
        print("\n======== Game Over ========")
        print("Calculating Final Net Worth...")

        final_standings = []
        for player in self.players:
            net_worth = player.calculate_net_worth(self.stocks)
            final_standings.append((player.name, net_worth))
            print(f"{player.name}: Final Net Worth ₹{net_worth:.2f}")

        final_standings.sort(key=lambda item: item[1], reverse=True)

        print("\n--- Final Rankings ---")
        for i, (name, net_worth) in enumerate(final_standings):
            print(f"{i+1}. {name} - ₹{net_worth:.2f}")

        print(f"\nWinner is: {final_standings[0][0]}!")


    def play(self):
        while self.current_session < self.num_sessions:
            self.play_session()
            # Optional: Pause or prompt before next session
            if self.current_session < self.num_sessions:
                input("\nPress Enter to start the next session...")

        self.end_game()


# --- Game Setup and Start ---
if __name__ == "__main__":
    num_players = 0
    while not 2 <= num_players <= 6:
        try:
            num_players_str = input("Enter the number of players (2-6): ")
            num_players = int(num_players_str)
            if not 2 <= num_players <= 6:
                print("Invalid number of players.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    player_names = []
    for i in range(num_players):
        name = input(f"Enter name for Player {i+1}: ")
        player_names.append(name)

    num_sessions = 0
    while num_sessions <= 0:
         try:
              num_sessions_str = input("Enter the number of sessions to play: ")
              num_sessions = int(num_sessions_str)
              if num_sessions <= 0:
                   print("Number of sessions must be positive.")
         except ValueError:
              print("Invalid input. Please enter a number.")


    game = Game(player_names, num_sessions)
    game.play()


Enter the number of players (2-6): 3
Enter name for Player 1: pratham
Enter name for Player 2: diti
Enter name for Player 3: niharika
Enter the number of sessions to play: 6

======== Starting Session 1/6 ========

Dealing 9 event cards and 1 special card to each player...

--- Starting Trading Round 1 ---

--- pratham's Turn (Cash: ₹600000.00) ---

Available Actions:
1. Buy Stock
2. Sell Stock
3. Use Special Card
4. View Portfolio
5. View Event Cards
6. End Turn
Enter your choice (1-6): 1

Available Stocks:
1. HDFC: ₹25.00 (Opening: ₹25.00)
2. Infosys: ₹80.00 (Opening: ₹80.00)
3. TCS: ₹40.00 (Opening: ₹40.00)
4. Reliance: ₹75.00 (Opening: ₹75.00)
5. ONGC: ₹55.00 (Opening: ₹55.00)
6. Wockhardt: ₹20.00 (Opening: ₹20.00)
Enter number of stock to buy: 2
How many shares of Infosys do you want to buy? 12000
pratham doesn't have enough cash to buy 12000 shares of Infosys.

Available Actions:
1. Buy Stock
2. Sell Stock
3. Use Special Card
4. View Portfolio
5. View Event Cards
6. End Turn
Ente

KeyboardInterrupt: Interrupted by user